# United Kingdom

In [ ]:
import datetime
from itertools import repeat

import openpyxl
import requests

import pandas as pd

In [ ]:
from weekly_mort.core import DOWNLOAD_DIR_BASE, PROCESSED_DIR_BASE

## Settings

In [ ]:
sub = 'United Kingdom'

down_dir = DOWNLOAD_DIR_BASE / sub
proc_dir = PROCESSED_DIR_BASE / sub
for _dir in [down_dir, proc_dir]:
    _dir.mkdir(exist_ok=True, parents=True)

In [ ]:
urls = {2020:'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fbirthsdeathsandmarriages%2fdeaths%2fdatasets%2fweeklyprovisionalfiguresondeathsregisteredinenglandandwales%2f2020/publishedweek132020.xlsx',
        2019:'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fbirthsdeathsandmarriages%2fdeaths%2fdatasets%2fweeklyprovisionalfiguresondeathsregisteredinenglandandwales%2f2019/publishedweek522019.xls',
        2018: 'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fbirthsdeathsandmarriages%2fdeaths%2fdatasets%2fweeklyprovisionalfiguresondeathsregisteredinenglandandwales%2f2018/publishedweek522018withupdatedrespiratoryrow.xls',
        2017: 'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fbirthsdeathsandmarriages%2fdeaths%2fdatasets%2fweeklyprovisionalfiguresondeathsregisteredinenglandandwales%2f2017/publishedweek522017.xls'
}

## Process 2020 deaths

In [ ]:
with open(down_dir/'deaths2020.xlsx', 'wb') as f:
    f.write(requests.get(urls[2020]).content)

wb = openpyxl.load_workbook(down_dir/'deaths2020.xlsx')
sheet = wb['Weekly figures 2020']

In [ ]:
def filter_row(row):
    return list(filter(lambda x: x is not None, row[2:]))

def get_row_contents(sheet, first_col_val, colcheck = 0):
    for row in sheet.values:
        if row[colcheck] == first_col_val:
            return filter_row(row)

In [ ]:
enddates = get_row_contents(sheet, 'Week ended')

deaths = get_row_contents(sheet, 'Total deaths, all ages')

resp_deaths = get_row_contents(sheet,
                               'Deaths where the underlying cause was respiratory disease (ICD-10 J00-J99)',
                               1)

assert len(deaths) == len(resp_deaths)

In [ ]:
total = pd.DataFrame({'Week':[*enddates[:len(deaths)], *enddates[:len(deaths)]],
                      'Condition': [*list(repeat('Total', len(deaths))),
                                    *list(repeat('Respiratory', len(deaths)))],
                      'Deaths':[*deaths, *resp_deaths]})
total['Age'], total['Year'], total['Region'], total['Gender'] = 'Total', 2020, 'Total', 'Total'

total.head()

,Week,Condition,Deaths,Age,Year,Region,Gender
0,2020-01-03,Total,12254,Total,2020,Total,Total
1,2020-01-10,Total,14058,Total,2020,Total,Total
2,2020-01-17,Total,12990,Total,2020,Total,Total
3,2020-01-24,Total,11856,Total,2020,Total,Total
4,2020-01-31,Total,11612,Total,2020,Total,Total


In [ ]:
AGE_LIST = ['Under 1 year', '01-14', '15-44', '45-64', '65-74', '75-84', '85+']

In [ ]:
def _process_section(row_gen, row, gender='Total', year=2020, enddates=enddates):
    dfs = []
    found_flag = False
    while True:
        if row[1] in AGE_LIST:
            found_flag = True
            deaths = filter_row(row)
            df = pd.DataFrame({'Week':enddates[: len(deaths)], 'Deaths': deaths})
            df['Age'], df['Year'], df['Gender'] = row[1], year, gender
            df['Condition'], df['Region'] = 'Total', 'Total'
            dfs.append(df)
        elif found_flag:
            break
        row = next(row_gen)
    return row, pd.concat(dfs)

In [ ]:
gen_ex = sheet.values

for row in gen_ex:
    if row[1] and ('Persons' in row[1]):
        break

_, out = _process_section(gen_ex, row)

assert len(out) % len(AGE_LIST) == 0

out.head()

,Week,Deaths,Age,Year,Gender,Condition,Region
0,2020-01-03,48,Under 1 year,2020,Total,Total,Total
1,2020-01-10,50,Under 1 year,2020,Total,Total,Total
2,2020-01-17,69,Under 1 year,2020,Total,Total,Total
3,2020-01-24,53,Under 1 year,2020,Total,Total,Total
4,2020-01-31,50,Under 1 year,2020,Total,Total,Total


In [ ]:
def get_age_breakdown(sheet, year=2020, enddates=enddates):
    row_gen = sheet.values

    dfs = []
    pflag, fflag, mflag = [False]*3
    row = next(row_gen)

    while True:
        if row[1]:
            if 'Persons' in row[1]:
                row, df = _process_section(row_gen, row, 'Total', year, enddates)
                dfs.append(df)
                pflag = True
            if 'Males' in row[1]:
                row, df = _process_section(row_gen, row, 'Male', year, enddates)
                dfs.append(df)
                mflag = True
            if 'Females' in row[1]:
                row, df = _process_section(row_gen, row, 'Female', year, enddates)
                dfs.append(df)
                fflag = True
        elif pflag and mflag and fflag:
            break
        row = next(row_gen)
    return pd.concat(dfs)

In [ ]:
byages = get_age_breakdown(sheet)

In [ ]:
def get_geographical(sheet, year=2020, enddates=enddates):

    found_flag = False
    dfs = []

    for row in sheet.values:
        if row[1] and ('Deaths by region of usual residence' in row[1]):
            found_flag = True
            continue
        elif not found_flag:
            continue

        if row[1] is None:
            break
        deaths = filter_row(row)
        df = pd.DataFrame({'Week':enddates[: len(deaths)], 'Deaths': deaths})
        df['Age'], df['Year'], df['Gender'] = 'Total', year, 'Total'
        df['Condition'], df['Region'] = 'Total', row[1]
        dfs.append(df)

    return pd.concat(dfs)

In [ ]:
bygeo = get_geographical(sheet)

In [ ]:
all2020 = pd.concat([total, byages, bygeo], sort=True)

assert not any(all2020.isna().sum())

cols = ['Week', 'Year', 'Age', 'Gender', 'Condition', 'Region', 'Deaths']
all2020 = all2020[cols]

all2020.head()

,Week,Year,Age,Gender,Condition,Region,Deaths
0,2020-01-03,2020,Total,Total,Total,Total,12254
1,2020-01-10,2020,Total,Total,Total,Total,14058
2,2020-01-17,2020,Total,Total,Total,Total,12990
3,2020-01-24,2020,Total,Total,Total,Total,11856
4,2020-01-31,2020,Total,Total,Total,Total,11612


In [ ]:
all2020.to_csv(proc_dir/'deaths2020.csv')

In [ ]:
all2020.Gender.value_counts()

Total     247
Female     91
Male       91
Name: Gender, dtype: int64

## Process Previous Years

In [ ]:
urls

{2020: 'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fbirthsdeathsandmarriages%2fdeaths%2fdatasets%2fweeklyprovisionalfiguresondeathsregisteredinenglandandwales%2f2020/publishedweek132020.xlsx',
 2019: 'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fbirthsdeathsandmarriages%2fdeaths%2fdatasets%2fweeklyprovisionalfiguresondeathsregisteredinenglandandwales%2f2019/publishedweek522019.xls',
 2018: 'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fbirthsdeathsandmarriages%2fdeaths%2fdatasets%2fweeklyprovisionalfiguresondeathsregisteredinenglandandwales%2f2018/publishedweek522018withupdatedrespiratoryrow.xls',
 2017: 'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fbirthsdeathsandmarriages%2fdeaths%2fdatasets%2fweeklyprovisionalfiguresondeathsregisteredinenglandandwales%2f2017/publishedweek522017.xls'}

In [ ]:
import xlrd
from openpyxl.workbook import Workbook

def cvt_xls_to_xlsx(src_file_path, dst_file_path):
    '''
    From https://stackoverflow.com/questions/9918646/how-to-convert-xls-to-xlsx/42574983#42574983
    '''
    book_xls = xlrd.open_workbook(src_file_path)
    book_xlsx = Workbook()

    sheet_names = book_xls.sheet_names()
    for sheet_index, sheet_name in enumerate(sheet_names):
        sheet_xls = book_xls.sheet_by_name(sheet_name)
        if sheet_index == 0:
            sheet_xlsx = book_xlsx.active
            sheet_xlsx.title = sheet_name
        else:
            sheet_xlsx = book_xlsx.create_sheet(title=sheet_name)

        for row in range(0, sheet_xls.nrows):
            for col in range(0, sheet_xls.ncols):
                sheet_xlsx.cell(row = row+1 , column = col+1).value = sheet_xls.cell_value(row, col)

    book_xlsx.save(dst_file_path)

In [ ]:
years = list(range(2017, 2020))

In [ ]:
for year in years:
    cvt_xls_to_xlsx(down_dir/f'deaths{year}.xls', down_dir/f'deaths{year}.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: '../_downloads/United Kingdom/deaths2017.xls'

In [ ]:
wbs = [openpyxl.load_workbook(down_dir/f'deaths{year}.xlsx') for year in years]

sheetnames = [f'Weekly figures {y}' for y in years]

sheets = [wb[shname] for wb, shname in zip(wbs, sheetnames)]

FileNotFoundError: [Errno 2] No such file or directory: '../_downloads/United Kingdom/deaths2017.xlsx'

In [ ]:
def conv2date(xldate, datemode=0):
    '''
    from https://stackoverflow.com/questions/1108428/how-do-i-read-a-date-in-excel-format-in-python
    '''
    # datemode: 0 for 1900-based, 1 for 1904-based
    return (
        datetime.datetime(1899, 12, 30)
        + datetime.timedelta(days=xldate + 1462 * datemode)
        )

In [ ]:
_conv_dates = lambda x: [conv2date(y) for y in x]

enddates = [_conv_dates(get_row_contents(sh, 'Week ended')) for sh in sheets]

In [ ]:
byage = [get_age_breakdown(sh, y, end) for sh, y, end in zip(sheets, years, enddates)]

bygeo = [get_geographical(sh, y, end) for sh, y, end in zip(sheets, years, enddates)]

In [ ]:
def get_totals(sheet, year, enddates):
    deaths = get_row_contents(sheet, 'Total deaths, all ages')
    
    resp_deaths = get_row_contents(sheet,
                               'All respiratory diseases (ICD-10 J00-J99)\nICD-10 v 2013 (IRIS)',
                               1)
    
    total = pd.DataFrame({'Week':[*enddates[:len(deaths)], *enddates[:len(deaths)]],
                          'Condition': [*list(repeat('Total', len(deaths))),
                                        *list(repeat('Respiratory', len(deaths)))],
                          'Deaths':[*deaths, *resp_deaths]})
    total['Age'], total['Year'], total['Region'], total['Gender'] = 'Total', 2020, 'Total', 'Total'
    return total

In [ ]:
totals = [get_totals(sh, y, end) for sh, y, end in zip(sheets, years, enddates)]

In [ ]:
all_dfs = [pd.concat([t, a, g], sort=True) for t, a, g in zip(totals, byage, bygeo)]

all_dfs = [df[cols] for df in all_dfs]

In [ ]:
for year, df in zip(years, all_dfs):
    df.to_csv(proc_dir/f'deaths{year}.csv')